Clustering Analysis Pipeline

Este notebook implementa e avalia três métodos de clustering:
- K-means
- Hierarchical Clustering (Bottom-up)
- Hierarchical Clustering (Top-down)

O pipeline executa os métodos de clustering e avalia os resultados para
diferentes números de clusters, salvando os resultados em arquivos CSV e
TXT, cada arquivo txt é nomeado a partir do resumo do cluster

Autor: [github.com/ArthurFachel]

Data: [28/12]

In [ ]:
import os
from utils.run import kmeans_run, hierarchical_bottom_top_run, hierarchical_top_bottom_run
from utils.evaluation import evaluate_top, evaluate_bottom, evaluate_kmeans
from dotenv import load_dotenv

load_dotenv()
folder = os.getenv("FOLDER_PATH")
print(f"Diretório de modelos: {folder}")

In [ ]:
def initialize_clustering_methods(i):
    """
    Inicializa e executa todos os métodos de clustering.
    
    Esta função executa três algoritmos de clustering diferentes com o número
    máximo de clusters especificado e retorna os resultados em um dicionário.
    
    Parameters
    ----------
    i : int
        Número máximo de clusters para inicializar os modelos
        
    Returns
    -------
    dict
        Dicionário contendo os DataFrames resultantes de cada método:
        - 'kmeans': Resultado do algoritmo K-means
        - 'bottom_top': Resultado do clustering hierárquico bottom-up
        - 'top_bottom': Resultado do clustering hierárquico top-down
        
    Examples
    --------
    >>> results = initialize_clustering_methods(11)
    >>> kmeans_df = results['kmeans']
    """
    df_kmeans = kmeans_run(max_clusters=i)
    df_bottom_top = hierarchical_bottom_top_run(max_clusters=i)
    df_top_bottom = hierarchical_top_bottom_run(max_clusters=i)
    
    return {
        "kmeans": (df_kmeans,),
        "bottom_top": (df_bottom_top),
        "top_bottom": (df_top_bottom,)
    }


Função de avaliação serve para quando se é uma pasta com resultados de diferentes modelos sobre o mesmo dataset

In [ ]:
def evaluate_and_save(method_name, evaluate_func, num_clusters, folder):
    """
    Avalia métodos de clustering e salva os resultados em arquivos CSV.
    
    Para cada número de clusters especificado, esta função:
    1. Executa a função de avaliação correspondente
    2. Salva os resultados em um arquivo CSV
    3. Armazena os resultados em um dicionário
    
    Parameters
    ----------
    method_name : str
        Nome do método de clustering ('kmeans', 'top_bottom', ou 'bottom_top')
    evaluate_func : callable
        Função de avaliação a ser executada (evaluate_kmeans, evaluate_top, ou evaluate_bottom)
    num_clusters : list of int
        Lista com os números de clusters a serem avaliados
    folder : str
        Caminho da pasta contendo os modelos salvos
        
    Returns
    -------
    dict
        Dicionário mapeando número de clusters para DataFrame de resultados
        Chave: número de clusters
        Valor: DataFrame com métricas de avaliação
        
    Examples
    --------
    >>> results = evaluate_and_save('kmeans', evaluate_kmeans, [6, 11], '/path/to/models')
    Salvo: kmeans_5.csv
    Salvo: kmeans_10.csv
    
    Notes
    -----
    - Os arquivos CSV são salvos com nome no formato: {method_name}_{count-1}.csv
    - O índice do DataFrame não é incluído no arquivo CSV
    """
    results = {}
    
    for count in num_clusters:
        result = evaluate_func(models_folder=folder, i=count)
        filename = f"{method_name}_{count-1}.csv"
        result.to_csv(filename, index=False)
        results[count] = result
        print(f"✓ Salvo: {filename}")
    
    return results

Defining how many clusters we want to run


In [ ]:
num_clusters = [6, 11]
initialize_clustering_methods(max(num_clusters))

In [ ]:
methods = {
        'top_bottom': evaluate_top,
        'bottom_top': evaluate_bottom,
        'kmeans': evaluate_kmeans
    }
    
all_results = {}
for method_name, evaluate_func in methods.items():
        all_results[method_name] = evaluate_and_save(
            method_name, 
            evaluate_func, 
            num_clusters, 
            folder
        )